In [1]:
import mne, os
import numpy as np
import pandas as pd 
from joblib import Parallel, delayed



In [8]:

#*****************************#
### HELPER FUNCTIONS ###
#*****************************#
def setup_paths(meg_dir, session):
    run_paths,event_paths = [],[]
    for file in os.listdir(f'{meg_dir}/ses-{str(session).zfill(2)}/meg/'):
        if file.endswith(".ds") and file.startswith("sub"):
            run_paths.append(os.path.join(f'{meg_dir}/ses-{str(session).zfill(2)}/meg/', file))
        if file.endswith("events.tsv") and file.startswith("sub"):
            event_paths.append(os.path.join(f'{meg_dir}/ses-{str(session).zfill(2)}/meg/', file))
    run_paths.sort()
    event_paths.sort()

    return run_paths, event_paths 

def read_raw(curr_path,session,run,participant):
    raw = mne.io.read_raw_ctf(curr_path,preload=True)
    # signal dropout in one run -- replacing values with median
    if participant == '1' and session == 11 and run == 4:  
        n_samples_exclude   = int(0.2/(1/raw.info['sfreq']))
        raw._data[:,np.argmin(np.abs(raw.times-13.4)):np.argmin(np.abs(raw.times-13.4))+n_samples_exclude] = np.repeat(np.median(raw._data,axis=1)[np.newaxis,...], n_samples_exclude, axis=0).T
    elif participant == '2' and session == 10 and run == 2: 
        n_samples_exclude = int(0.2/(1/raw.info['sfreq']))
        raw._data[:,np.argmin(np.abs(raw.times-59.8)):np.argmin(np.abs(raw.times-59.8))+n_samples_exclude] = np.repeat(np.median(raw._data,axis=1)[np.newaxis,...], n_samples_exclude, axis=0).T

    raw.drop_channels('MRO11-1609')
        
    return raw

def read_events(event_paths,run,raw):
    # load event file that has the corrected onset times (based on optical sensor and replace in the events variable)
    event_file = pd.read_csv(event_paths[run],sep='\t')
    event_file.value.fillna(999999,inplace=True)
    events = mne.find_events(raw, stim_channel=trigger_channel,initial_event=True)
    events = events[events[:,2]==trigger_amplitude]
    events[:,0] = event_file['sample']
    events[:,2] = event_file['value']
    return events

def concat_epochs(raw, events, epochs):
    if epochs:
        epochs_1 = mne.Epochs(raw, events, tmin = pre_stim_time, tmax = post_stim_time, picks = 'mag',baseline=None)
        epochs_1.info['dev_head_t'] = epochs.info['dev_head_t']
        epochs = mne.concatenate_epochs([epochs,epochs_1])
    else:
        epochs = mne.Epochs(raw, events, tmin = pre_stim_time, tmax = post_stim_time, picks = 'mag',baseline=None)
    return epochs

def baseline_correction(epochs):
    baselined_epochs = mne.baseline.rescale(data=epochs.get_data(),times=epochs.times,baseline=(None,0),mode='zscore',copy=False)
    epochs = mne.EpochsArray(baselined_epochs, epochs.info, epochs.events, epochs.tmin,event_id=epochs.event_id)
    return epochs

def stack_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution):
    for epochs in session_epochs:
        epochs.info['dev_head_t'] = session_epochs[0].info['dev_head_t']
    all_epochs = mne.concatenate_epochs(epochs_list = session_epochs, add_offset=True)
    all_epochs.metadata = pd.read_csv(f'{sourcedata_dir}/sample_attributes_P{str(participant)}.csv')
    all_epochs.decimate(decim=(1200/output_resolution))
    all_epochs.save(f'{preproc_dir}/preprocessed_P{str(participant)}-epo.fif', overwrite=True)
    print(all_epochs.info)

def save_per_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution, sess_list):
    start_id = 0
    for sess, epochs in zip(sess_list, session_epochs):
        end_id = start_id + len(epochs)
        metadata = pd.read_csv(f'{sourcedata_dir}/sample_attributes_P{str(participant)}.csv')
        epochs.metadata = metadata.iloc[start_id:end_id]
        epochs.decimate(decim=(1200/output_resolution))
        savefile = f'{preproc_dir}/preprocessed_P{str(participant)}-epo-sess{sess}.fif'
        epochs.save(savefile, overwrite=True)
        start_id = end_id
    print(savefile)
    
    
#*****************************#
### FUNCTION TO RUN PREPROCESSING ###
#*****************************#
def run_preprocessing(meg_dir,session,participant):
    epochs = []
    run_paths, event_paths = setup_paths(meg_dir, session)
    for run, curr_path in enumerate(run_paths):
        raw = read_raw(curr_path,session,run, participant)
        events = read_events(event_paths,run,raw)
        raw.filter(l_freq=l_freq,h_freq=h_freq)
        epochs = concat_epochs(raw, events, epochs)
        epochs.drop_bad()
    print(epochs.info)
    epochs = baseline_correction(epochs)
    return epochs


In [3]:
#*****************************#
### PARAMETERS ###
#*****************************#

n_sessions                  = 12
trigger_amplitude           = 64
l_freq                      = 0.1
h_freq                      = 40
pre_stim_time               = -0.1
post_stim_time              = 1.3
std_deviations_above_below  = 4
output_resolution           = 200
trigger_channel             = 'UPPT001'



class Args():
        bids_dir = '/work/project/MEG_GOD/yainoue/things/MEG'
        image_dir = '/work/project/MEG_GOD/yainoue/things/Images'
        participant = 1

args = Args()

bids_dir                    = args.bids_dir
participant                 = args.participant
meg_dir                     = f'{bids_dir}/sub-BIGMEG{participant}/'
sourcedata_dir              = f'{bids_dir}/sourcedata/'
preproc_dir                 = f'{bids_dir}/derivatives/preprocessed/'
if not os.path.exists(preproc_dir):
    os.makedirs(preproc_dir)
    
session_epochs = []
for session in range(1, n_sessions+1):
    epoch = run_preprocessing(meg_dir,session,participant)
    session_epochs.append(epoch)

# stack_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution)

ds directory : /work/project/MEG_GOD/yainoue/things/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-01_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.92   77.00    0.00 mm <->   -0.92   77.00    0.00 mm (orig :  -40.06   68.57 -237.90 mm) diff =    0.000 mm
       0.92  -77.00    0.00 mm <->    0.92  -77.00    0.00 mm (orig :   70.03  -39.14 -237.69 mm) diff =    0.000 mm
      79.58    0.00    0.00 mm <->   79.58   -0.00    0.00 mm (orig :   71.23   70.87 -233.83 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate locations for 3 HPI coils added
    Measurement info composed.
Finding samples for /work/project/MEG_GOD/yainoue/things/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-01_meg.ds/sub-BIGMEG1_ses-01_task-main_run-01_meg.meg4: 
    System clock channel is available, checking which samples

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    8.6s finished


Not setting metadata
226 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 226 events and 1681 original time points ...
0 bad epochs dropped
ds directory : /work/project/MEG_GOD/yainoue/things/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-02_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.71   77.07    0.00 mm <->   -0.71   77.07    0.00 mm (orig :  -40.27   68.70 -237.97 mm) diff =    0.000 mm
       0.71  -77.07    0.00 mm <->    0.71  -77.07    0.00 mm (orig :   69.91  -39.09 -237.96 mm) diff =    0.000 mm
      79.46    0.00    0.00 mm <->   79.46   -0.00    0.00 mm (orig :   70.83   71.02 -233.95 mm) diff =    0.000 mm
    Coordinate transformations established.
    Polhemus data for 3 HPI coils added
    Device coordinate lo

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    8.9s finished


Not setting metadata
226 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 226 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 226 events and 1681 original time points ...
Using data from preloaded Raw for 226 events and 1681 original time points ...
Not setting metadata
452 matching events found
No baseline correction applied
ds directory : /work/project/MEG_GOD/yainoue/things/sub-BIGMEG1/ses-01/meg/sub-BIGMEG1_ses-01_task-main_run-03_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
      -0.64   77.10    0.00 mm <->   -0.64   77.10   -0.00 mm (orig :  -40.41   68.84 -238.00 mm) diff =    0.000 mm
       0.64  -77.10    0.00 mm <->    0.64  -77.10    0.00 mm (orig :   69.73  -39.08 -238.46 mm) diff =  

...

Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 225 events and 1681 original time points ...
Not setting metadata
1804 matching events found
No baseline correction applied
ds directory : /work/project/MEG_GOD/yainoue/things/sub-BIGMEG1/ses-12/meg/sub-BIGMEG1_ses-12_task-main_run-09_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       0.41   77.53    0.00 mm <->    0.41   77.53    0.00 mm (orig :  -40.38   68.73 -238.92 mm) diff =    0.000 mm
      -0.41  -77.53    0.00 mm <->   -0.41  -77.53    0.00 mm (orig :   70.70  -39.45 -238.37 mm) diff =    0.000 mm
      76.77    0.00    0.00 mm <->   76.77   -0.00   -0.00 mm (orig

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    8.4s finished


Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 225 events and 1681 original time points ...
Not setting metadata
2029 matching events found
No baseline correction applied
ds directory : /work/project/MEG_GOD/yainoue/things/sub-BIGMEG1/ses-12/meg/sub-BIGMEG1_ses-12_task-main_run-10_meg.ds
    res4 data read.
    hc data read.
    Separate EEG position data file read.
    Quaternion matching (desired vs. transformed):
       0.41   77.55    0.00 mm <->    0.41   77.55    0.00 mm (orig :  -40.43   68.76 -238.74 mm) diff =    0.000 mm
      -0.41  -77.55    0.00 mm <->   -0.41  -77.55    0.00 mm (orig :   70.70  -39.44 -238.28 mm) diff =    0.000 mm
      76.78    0.00    0.00 mm <->   76.78   -0.00    0.00 mm (orig

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   3 out of   3 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done   4 out of   4 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=1)]: Done 299 out of 299 | elapsed:    8.9s finished


Not setting metadata
225 matching events found
Removing 5 compensators from info because not all compensation channels were picked.
No baseline correction applied
0 projection items activated
Using data from preloaded Raw for 225 events and 1681 original time points ...
0 bad epochs dropped
Using data from preloaded Raw for 225 events and 1681 original time points ...
Not setting metadata
2254 matching events found
No baseline correction applied
<Info | 15 non-empty values
 bads: []
 ch_names: MLC11-1609, MLC12-1609, MLC13-1609, MLC14-1609, MLC15-1609, ...
 chs: 271 Magnetometers
 ctf_head_t: CTF/4D/KIT head -> head transform
 custom_ref_applied: False
 dev_ctf_t: MEG device -> CTF/4D/KIT head transform
 dev_head_t: MEG device -> head transform
 dig: 3 items (3 Cardinal)
 experimenter: hebart
 highpass: 0.1 Hz
 hpi_results: 1 item (list)
 lowpass: 40.0 Hz
 meas_date: 2019-07-12 12:14:00 UTC
 meas_id: 4 items (dict)
 nchan: 271
 projs: []
 sfreq: 1200.0 Hz
 subject_info: 1 item (dict)
>

In [9]:
save_per_sessions(sourcedata_dir,preproc_dir,participant,session_epochs,output_resolution, list(range(1, n_sessions+1)))

Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess1.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess2.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


Adding metadata with 18 columns


...

Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess11.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


Adding metadata with 18 columns


/tmp/ipykernel_110985/778953778.py:71: RuntimeWarning: This filename (/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess12.fif) does not conform to MNE naming conventions. All epochs files should end with -epo.fif, -epo.fif.gz, _epo.fif or _epo.fif.gz
  epochs.save(savefile, overwrite=True)


/work/project/MEG_GOD/yainoue/things/derivatives/preprocessed//preprocessed_P1-epo-sess12.fif


In [11]:
total_n = 0
for epochs in session_epochs:
    print(len(epochs))
    total_n += len(epochs)
print('total: ', total_n)

2254
2254
2254
2254
2254
2254
2254
2254
2254
2254
2254
2254
total:  27048


In [13]:
27048 - 22448

4600

In [37]:
2254 * 12

27048

In [26]:
epoch.metadata.keys()

Index(['trial_type', 'image_nr', 'category_nr', 'exemplar_nr', 'test_image_nr',
       'things_category_nr', 'things_image_nr', 'things_exemplar_nr',
       'image_path', 'onset', 'image_on', 'image_off', 'responded', 'key_id',
       'key_time', 'RT', 'session_nr', 'run_nr'],
      dtype='object')

In [27]:
exp_df = epoch.metadata.query('trial_type=="exp"')
test_df = epoch.metadata.query('trial_type=="test"')

In [39]:
NUM_TRAIN = 0
NUM_TEST = 0
for epoch in session_epochs:
    exp_df = epoch.metadata.query('trial_type=="exp"')
    test_df = epoch.metadata.query('trial_type=="test"')
    catch_df = epoch.metadata.query('trial_type=="catch"') # artificial image
    print('total: ', len(epoch.metadata), 'train: ', len(exp_df), 'test: ', len(test_df), 'catch: ', len(catch_df))
    print('train max', exp_df['image_nr'].max()) # 22248終わり
    print('train min', exp_df['image_nr'].min()) # 1始まり
    print('test max', test_df['image_nr'].max()) # 22648終わり
    print('test min', test_df['image_nr'].min()) # 22449始まり
    print('catch max', catch_df['image_nr'].max()) # 22448 終わり
    print('catch min', catch_df['image_nr'].min()) # 22249始まり
    NUM_TRAIN +=  len(exp_df)
    NUM_TEST += len(test_df)
print('TRAIN: ', NUM_TRAIN)
print('TEST: ', NUM_TEST)

total:  2254 train:  1854 test:  200 catch:  200
train max 22246
train min 10
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22238
train min 2
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22237
train min 1
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22248
train min 12
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22243
train min 7
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22242
train min 6
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854 test:  200 catch:  200
train max 22241
train min 5
test max 22648
test min 22449
catch max 22448
catch min 22249
total:  2254 train:  1854

In [38]:
epoch.metadata['trial_type'].unique()

array(['exp', 'test', 'catch'], dtype=object)